# 10.3 Apply：General split-apply-combine（应用：通用的分割-应用-合并）

>general-purpose: 可以理解为通用，泛用。

>例子：在计算机软件中，通用编程语言(General-purpose programming language )指被设计为各种应用领域服务的编程语言。通常通用编程语言不含有为特定应用领域设计的结构。

>相对而言，特定域编程语言就是为某一个特定的领域或应用软件设计的编程语言。比如说，LaTeX就是专门为排版文献而设计的语言。

最通用的GroupBy(分组)方法是apply，这也是本节的主题。如下图所示，apply会把对象分为多个部分，然后将函数应用到每一个部分上，然后把所有的部分都合并起来：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/81f9f.png)

返回之前提到的tipping数据集，假设我们想要根据不同组（group），选择前5个tip_pct值最大的。首先，写一个函数，函数的功能为在特定的列，选出有最大值的行:

In [2]:
import numpy as np
import pandas as pd

In [3]:
tips = pd.read_csv('../../examples/tips.csv')

In [4]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [5]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [6]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [7]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


现在，如果我们按smoker分组，然后用apply来使用这个函数，我们能得到下面的结果：

In [8]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

我们来解释下上面这一行代码发生了什么。这里的top函数，在每一个DataFrame中的行组（row group）都被调用了一次，然后各自的结果通过pandas.concat合并了，最后用组名（group names）来标记每一部分。（译者：可以理解为，我们先按smoker这一列对整个DataFrame进行了分组，一共有No和Yes两组，然后对每一组上调用了top函数，所以每一组会返还5行作为结果，最后把两组的结果整合起来，一共是10行）。

最后的结果是有多层级索引（hierarchical index）的，而且这个多层级索引的内部层级（inner level）含有来自于原来DataFrame中的索引值（index values）（译者：即在smoker为No的这一组，No本身是一个索引，它的内层索引是88, 185, 51, 149, 232这五个行索引，这五个内部层级是来自于原始DataFrame的）。

如果传递一个函数给apply，可以在函数之后，设定其他一些参数：

In [9]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

除了上面这些基本用法，要想用好apply可能需要一点创新能力。毕竟传给这个函数的内容取决于我们自己，而最终的结果只需要返回一个pandas对象或一个标量。这一章的剩余部分主要介绍如何解决在使用groupby时遇到的一些问题。

可以试一试在GroupBy对象上调用describe：

In [10]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [12]:
result
# 更新后的 pandas 在类似输出时是二维表

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [13]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

在GroupBy内部，当我们想要调用一个像describe这样的函数的时候，其实相当于下面的写法：

    f = lambda x: x.describe()
    grouped.apply(f)
    
# 1 Suppressing the Group Keys（抑制组键）

在接下来的例子，我们会看到作为结果的对象有一个多层级索引（hierarchical index），这个多层级索引是由原来的对象中，组键（group key）在每一部分的索引上得到的。我们可以在groupby函数中设置group_keys=False来关闭这个功能：

In [14]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


# 2 Quantile and Bucket Analysis（分位数与桶分析）

在第八章中，我们介绍了pandas的一些工具，比如cut和qcut，通过设置中位数，切割数据为buckets with bins(有很多箱子的桶)。

> 这里bucket我翻译为桶，可以理解为像group一样的概念，一个组内有不同的bins。而关于bins（箱）的部分，可以回顾看一下7.2

把函数通过groupby整合起来，可以在做桶分析或分位数分析的时候更方便。假设一个简单的随机数据集和一个等长的桶类型（bucket categorization），使用cut：

In [15]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                       'data2': np.random.randn(1000)})
frame.head()

,data1,data2
0,-1.392165,-1.794926
1,-0.072331,-1.196881
2,1.539681,-1.775704
3,0.647617,1.168386
4,-1.211960,0.334770


In [16]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.668, -0.164]
1      (-0.164, 1.34]
2       (1.34, 2.844]
3      (-0.164, 1.34]
4    (-1.668, -0.164]
5    (-1.668, -0.164]
6    (-1.668, -0.164]
7      (-0.164, 1.34]
8       (1.34, 2.844]
9    (-1.668, -0.164]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.178, -1.668] < (-1.668, -0.164] < (-0.164, 1.34] < (1.34, 2.844]]

cut返回的Categorical object（类别对象）能直接传入groupby。所以我们可以在data2列上计算很多统计值：

In [17]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [18]:
grouped = frame.data2.groupby(quartiles)

In [19]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.178, -1.668]",-2.426985,2.018192,57.0,0.076522
"(-1.668, -0.164]",-2.700738,3.432337,390.0,-0.008452
"(-0.164, 1.34]",-3.147846,3.078179,458.0,0.003899
"(1.34, 2.844]",-2.648108,2.491551,95.0,0.051260


也有相同长度的桶（equal-length buckets）；想要按照样本的分位数得到相同长度的桶，用qcut。这里设定labels=False来得到分位数的数量：

In [20]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

译者：上面的代码是把frame的data1列分为10个bin，每个bin都有相同的数量。因为一共有1000个样本，所以每个bin里有100个样本。grouping保存的是每个样本的index以及其对应的bin的编号。

In [21]:
grouped = frame.data2.groupby(grouping)

In [22]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.476626,2.018192,100.0,0.016125
1,-2.700738,3.432337,100.0,-0.154557
2,-2.271151,2.294226,100.0,0.061269
3,-2.597092,2.810862,100.0,0.100332
4,-2.126149,2.011462,100.0,0.009309
5,-1.950020,2.197623,100.0,-0.006610
6,-3.147846,1.935005,100.0,-0.118148
7,-2.763037,1.736598,100.0,0.075650
8,-2.612539,3.078179,100.0,0.027622


对于pandas的Categorical类型，会在第十二章做详细介绍。

# 3 Example: Filling Missing Values with Group-Specific Values（例子：用组特异性值来填充缺失值）

在处理缺失值的时候，一些情况下我们会直接用dropna来把缺失值删除，但另一些情况下，我们希望用一些固定的值来代替缺失值，而fillna就是用来做这个的，例如，这里我们用平均值mean来代替缺失值NA：

In [23]:
s = pd.Series(np.random.randn(6))

In [24]:
s[::2] = np.nan

In [25]:
s

0         NaN
1    1.201570
2         NaN
3    1.620979
4         NaN
5   -0.599705
dtype: float64

In [26]:
s.fillna(s.mean())

0    0.740948
1    1.201570
2    0.740948
3    1.620979
4    0.740948
5   -0.599705
dtype: float64

假设我们想要给每一组填充不同的值。一个方法就是对数据分组后，用apply来调用fillna，在每一个组上执行一次。这里有一些样本是把美国各州分为西部和东部：

In [27]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [28]:
group_key = ['East'] * 4 + ['West'] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [29]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.166060
New York      0.934047
Vermont       0.629597
Florida      -0.808614
Oregon        0.272138
Nevada       -1.945503
California   -0.357273
Idaho        -1.408868
dtype: float64

我们令data中某些值为缺失值：

In [30]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.166060
New York      0.934047
Vermont            NaN
Florida      -0.808614
Oregon        0.272138
Nevada             NaN
California   -0.357273
Idaho              NaN
dtype: float64

In [31]:
data.groupby(group_key).mean()

East   -0.346876
West   -0.042568
dtype: float64

然后我们可以用每个组的平均值来填充NA：

In [32]:
fill_mean = lambda g: g.fillna(g.mean())

In [33]:
data.groupby(group_key).apply(fill_mean)

Ohio         -1.166060
New York      0.934047
Vermont      -0.346876
Florida      -0.808614
Oregon        0.272138
Nevada       -0.042568
California   -0.357273
Idaho        -0.042568
dtype: float64

在另外一些情况下，我们可能希望提前设定好用于不同组的填充值。因为group有一个name属性，我们可以利用这个：

In [34]:
fill_values = {'East': 0.5, 'West': -1}

In [35]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [36]:
data.groupby(group_key).apply(fill_func)

Ohio         -1.166060
New York      0.934047
Vermont       0.500000
Florida      -0.808614
Oregon        0.272138
Nevada       -1.000000
California   -0.357273
Idaho        -1.000000
dtype: float64

# 4 Example: Random Sampling and Permutation（例子：随机抽样和排列）

假设我们想要从一个很大的数据集里随机抽出一些样本，这里我们可以在Series上用sample方法。为了演示，这里先创建一副模拟的扑克牌：

In [37]:
# Hearts红桃，Spades黑桃，Clubs梅花，Diamonds方片
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

这样我们就得到了一个长度为52的Series，索引（index）部分是牌的名字，对应的值为牌的点数，这里的点数是按Blackjack（二十一点）的游戏规则来设定的。

> Blackjack（二十一点）: 2点至10点的牌以牌面的点数计算，J、Q、K 每张为10点，A可记为1点或为11点。这里为了方便，我们只把A记为1点。

In [38]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

现在，就像我们上面说的，随机从牌组中抽出5张牌：

In [39]:
def draw(deck, n=5):
    return deck.sample(n)

In [40]:
draw(deck)

6C      6
6H      6
3C      3
10H    10
QS     10
dtype: int64

假设我们想要从每副花色中随机抽取两张，花色是每张牌名字的最后一个字符（即H, S, C, D），我们可以根据花色分组，然后使用apply：

In [41]:
get_suit = lambda card: card[-1] # last letter is suit

In [42]:
deck.groupby(get_suit).apply(draw, n=2)

C  AC      1
   6C      6
D  6D      6
   3D      3
H  10H    10
   8H      8
S  6S      6
   QS     10
dtype: int64

另外一种写法：

In [43]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C     5
JC    10
KD    10
8D     8
JH    10
QH    10
JS    10
6S     6
dtype: int64

# 5 Example: Group Weighted Average and Correlation（例子：组加权平均和相关性）

在groupby的split-apply-combine机制下，DataFrame的两列或两个Series，计算组加权平均（Group Weighted Average）是可能的。这里举个例子，下面的数据集包含组键，值，以及权重：

In [44]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.363393,0.491945
1,a,0.986890,0.337689
2,a,0.345090,0.676232
3,a,0.511157,0.014284
4,b,0.056615,0.214920
5,b,-0.522177,0.041985
6,b,0.587426,0.580789
7,b,-1.078869,0.958242


按category分组来计算组加权平均：

In [45]:
grouped = df.groupby('category')

In [46]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [47]:
grouped.apply(get_wavg)

category
a    0.259944
b   -0.391107
dtype: float64

另一个例子，考虑一个从Yahoo！财经上得到的经济数据集，包含一些股票交易日结束时的股价，以及S&P 500指数(即SPX符号)：

>标准普尔500指数英文简写为S&P 500 Index，是记录美国500家上市公司的一个股票指数。这个股票指数由标准普尔公司创建并维护。

>标准普尔500指数覆盖的所有公司，都是在美国主要交易所，如纽约证券交易所、Nasdaq交易的上市公司。与道琼斯指数相比，标准普尔500指数包含的公司更多，因此风险更为分散，能够反映更广泛的市场变化。

In [48]:
close_px = pd.read_csv('../../examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)

In [49]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [50]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


一个比较有意思的尝试是计算一个DataFrame，包括与SPX这一列逐年日收益的相关性（计算百分比变化）。一个可能的方法是，我们先创建一个能计算不同列相关性的函数，然后拿每一列与SPX这一列求相关性：

In [51]:
spx_corr = lambda x: x.corrwith(x['SPX'])

然后我们通过pct_change在close_px上计算百分比的变化：

In [52]:
rets = close_px.pct_change().dropna()

最后，我们按年来给这些百分比变化分组，年份可以从每行的标签中通过一个一行函数提取，然后返回的结果中，用datetime标签来表示年份：

In [53]:
get_year = lambda x: x.year

In [54]:
by_year = rets.groupby(get_year)

In [55]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


我们也可以计算列内的相关性。这里我们计算苹果和微软每年的相关性：

In [56]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 6 Example: Group-Wise Linear Regression（例子：组对组的线性回归）

就像上面介绍的例子，使用groupby可以用于更复杂的组对组统计分析，只要函数能返回一个pandas对象或标量。例如，我们可以定义regress函数（利用statsmodels库），在每一个数据块（each chunk of data）上进行普通最小平方回归（ordinary least squares (OLS) regression）计算：

In [57]:
import statsmodels.api as sm

In [58]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

现在，按年用苹果AAPL在标普SPX上做线性回归：

In [59]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
